In [6]:
!pip install -q cassio langchain openai tiktoken langchain_community datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00


import the pakcaged we will need

In [5]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

In [7]:
from datasets import load_dataset
# with cassio, engine poewring astra db integration in langchain. initalize the db connection
import cassio

In [ ]:
!pip install pyPDF2

In [9]:
from PyPDF2 import PdfReader

adding astra db coonection details and open ai key

In [10]:
ASTRA_DB_APPLICATION_TOKEN=""
ASTRA_DB_ID=""

OPENAI_API_KEY=""

In [11]:
pdfreader = PdfReader("resume.pdf")

In [14]:
from typing_extensions import Concatenate
raw_text=""
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

In [ ]:
raw_text

In [15]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [16]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

<ipython-input-16-799b35011f84>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=OPENAI_API_KEY)


In [18]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [19]:
astra_vector_store = Cassandra(
    embedding=embeddings,
    table_name="resume",
    session=None,
    keyspace=None
)

In [20]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(separator="\n",chunk_size=800, chunk_overlap=200, length_function=len)
texts = text_splitter.split_text(raw_text)

In [22]:
texts[:50]

['Piyush Yadav\nBhopal, Madhya Pradesh, India, 462008\n♂phone+91 8827413831 /envel⌢pepiyushyadav0191@gmail.com /linkedinlinkedin.com/in/piyushyadav0191 /product-huntPortfolio\nEducation\nBarkatullah University Institute of Technology [BUIT] Nov. 2020 – June 2024\nBachelor of Technology in Information Technology Bhopal, Madhya Pradesh\nExperience\nCodemate AI Sep 2023 – Sep 2024\nSDE - I - Full Stack Noida, UP\n•Designed and implemented full-stack services using Next.Js, Node.js, Flask, and Azure, enhancing the functionality and\nreliability of the system.\n•Collaborated with cross-functional teams to create a user-friendly dashboard, integrating front-end with Generative AI\nModels and ensuring seamless back-end functionality.',
 'reliability of the system.\n•Collaborated with cross-functional teams to create a user-friendly dashboard, integrating front-end with Generative AI\nModels and ensuring seamless back-end functionality.\n•Conducted regular code reviews, providing constructive 

In [25]:
astra_vector_store.add_texts(texts[:50])
print("Inserted %i headlines"% len(texts[:50]))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 6 headlines


In [35]:
first_question = True
while True:
  if first_question:
    query_text = input("\n Enter your questionn \n").strip()
  else:
    query_text= input("\n Whats your next question")

  if query_text.lower() == 'quit':
    break
  if query_text == "":
    continue

  first_question = False

  print("\n Question: \"%s\"" % query_text)
  answer = astra_vector_index.query(query_text, llm=llm).strip()
  print("Answer: \"%s\"" % answer)

  print("First documents by RELEVANCE")
  for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
    print("    [%0.4f] \"%s\"" % (score, doc.page_content[:84]))


KeyboardInterrupt: Interrupted by user